In [1]:
import os
import sys

sys.path.append('/home/robinm/phd/problog/')

In [2]:
from problog.program import PrologString
from problog.engine import DefaultEngine
from problog.logic import Term, Var
from problog.prolog_engine.engine_prolog import EngineProlog
from problog.formula import LogicFormula, LogicDAG
from graphviz import Source
from problog.cycles import break_cycles
from problog.sdd_formula import SDD
from time import time

from problog.prolog_engine.swi_program import SWIProgram

In [3]:
file='''
0.5::edge(1,2).
0.6::edge(1,3).
0.9::edge(2,4).
0.9::edge(2,5).
0.6::edge(3,6).
0.9::edge(4,6).
0.9::edge(5,6).
path(X,Y) :- edge(X,Y).
path(X,Y) :- edge(Z,Y), path(X,Z).
query(path(1,6)).
'''

In [4]:
program = PrologString(file)
engine = EngineProlog()
db = engine.prepare(program)
swiprogram = SWIProgram(db)
for l in swiprogram.get_lines():
    print(l+'.')

rule(path(A1,A2),[edge(A1,A2)]).
rule(path(A1,A2),[edge(A3,A2),path(A1,A3)]).
fact(0.5,edge(1,2),1).
fact(0.6,edge(1,3),2).
fact(0.9,edge(2,4),3).
fact(0.9,edge(2,5),4).
fact(0.6,edge(3,6),5).
fact(0.9,edge(4,6),6).
fact(0.9,edge(5,6),7).
fact(1.0,query(path(1,6)),8).


# Large graph

In [5]:
N=50000
file='''
0.6::edge(start,b).
0.6::edge(b,end).
0.5::edge(start,c).
0.9::edge(c,d).
0.9::edge(d,end).
0.9::edge(c,e).
0.9::edge(e,end).
{}
path(X,Y) :- edge(X,Y).
path(X,Y) :- edge(Z,Y), path(X,Z).
query(path(start,end)).
'''
extra_edges = ['0.01::edge(start,{}).'.format(i) for i in range(N)]
extra_edges += ['0.01::edge({},{}).'.format(i,i+N) for i in range(N)]
extra_edges += ['0.01::edge({},end).'.format(i+N) for i in range(N)]
file=file.format('\n'.join(extra_edges))
#print(file)
program = PrologString(file)



# Exact inference

In [6]:
engine = DefaultEngine()
db = engine.prepare(program)

In [7]:
start = time()
ground = engine.ground_all(db, target=LogicFormula())
print('Grounded in {} seconds'.format(time()-start))
start = time()
ac = SDD.create_from(ground)
print('Compiled in {} seconds'.format(time()-start))
start = time()
p = ac.evaluate()
print('Evaluated in {} seconds'.format(time()-start))
print('P = ',p)

Grounded in 73.86713457107544 seconds
Compiled in 11.870221138000488 seconds
Evaluated in 2.1127631664276123 seconds
P =  {path(start,end): 0.6846179897324968}


# K-best

In [8]:

engine = EngineProlog()
db = engine.prepare(program)
swiprogram = SWIProgram(db)
# for l in swiprogram.get_lines():
#     print(l+'.')

In [9]:
for k in range(1,5):
    print('{} proofs'.format(k))
    start = time()
    ground = engine.ground_all(db, target=LogicFormula(), k=k)
    print('    Grounded in {} seconds'.format(time()-start))
    start = time()
    ac = SDD.create_from(ground)
    print('    Compiled in {} seconds'.format(time()-start))
    start = time()
    p = ac.evaluate()
    print('    Evaluated in {} seconds'.format(time()-start))
    print('P = ',p)

1 proofs
    Grounded in 8.01073670387268 seconds
    Compiled in 0.5855984687805176 seconds
    Evaluated in 0.00020933151245117188 seconds
P =  {path(start,end): 0.405}
2 proofs
    Grounded in 7.71810507774353 seconds
    Compiled in 0.03845667839050293 seconds
    Evaluated in 0.00016307830810546875 seconds
P =  {path(start,end): 0.48195000000000005}
3 proofs
    Grounded in 7.764331102371216 seconds
    Compiled in 0.0008828639984130859 seconds
    Evaluated in 0.00017499923706054688 seconds
P =  {path(start,end): 0.668448}
4 proofs
    Grounded in 8.305662155151367 seconds
    Compiled in 0.0390164852142334 seconds
    Evaluated in 0.0002052783966064453 seconds
P =  {path(start,end): 0.668448331552}
